In [1]:
#Este es el script en el que se maneja el envio de eventos kafka
#Se crea un producer de kafka y se obtienen datos para enviar

"""
FUNCIONAL
"""
# kafka/bin/zookeeper-server-start.sh kafka/config/zookeeper.properties
# kafka/bin/kafka-server-start.sh kafka/config/server.properties
# kafka/bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic RF

# kafka/bin/kafka-topics.sh --delete --zookeeper localhost:2181 --topic RF
 
# kafka/bin/kafka-topics.sh --list --zookeeper localhost:2181
# kafka/bin/kafka-console-consumer.sh --bootstrap-server 192.168.1.101:9092 --topic RF --from-beginning

# ./elasticsearch/bin/elasticsearch
# ./kibana/bin/kibana
# curl -XDELETE 'http://localhost:9200/rf*'

import findspark
findspark.init()

import time

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

from kafka import KafkaProducer, TopicPartition
import uuid

In [2]:
#Se crea una sescion de spark
#Importante tras ejecucion hacer sc.stop()

sc = SparkContext('local')
spark = SparkSession(sc)

In [3]:
base_path='../../'

In [4]:
#Se define el esquema que va a tener el data frame

schema = StructType([
    StructField('time', StringType(), True),
    StructField('signal', StringType(), True),
    StructField('freq', StringType(), True),
    StructField('mod', StringType(), True),
    StructField('payload', StringType(), True),
])

In [5]:
df = spark.read.schema(schema).option("mode", "DROPMALFORMED").json(f'{base_path}datasets/dataset_rf_v2.json')

In [6]:
# Añadimos metadatos para simular eventos del sensor

df = df.withColumn('data', struct(col('*')))
df = df.withColumn('version', lit('1.0'))
df = df.withColumn('id' , lit('f0c48ba4-387d-11ea-a137-2e728ce88126'))
df = df.withColumn('type', lit('RF'))
df = df.withColumn('event', lit('DATA'))

In [ ]:
# Valores atipicos dataset

'''
df = df.filter(( df.payload == '40e888e8e88800000008e888e8') | 
                 ( df.payload == '0d111d1d11000000011d111d1d') | 
                 ( df.payload == '00e8e888e8e88800000008e888') |
                 ( df.payload == 'a5004143e93c7fffffffffffff' ))
'''

In [7]:
# Creamos productor de eventos Kafka

ip_server = '138.4.7.158:9092'
kafka_topic = 'RF-DATA'
c = 0
sleep = 20
seed = 1

while True:
    df_sample = df.sample(fraction = 0.1, seed = seed)
    df_sample = df_sample.withColumn('time', unix_timestamp().cast(StringType()))
    
    prediction_features = df_sample.select('version','time','id','type','event','data')

    if type(kafka_topic) == bytes:
        kafka_topic = kafka_topic.decode('utf-8')
    producer = KafkaProducer(bootstrap_servers=[ip_server],api_version=(0,10))
    PREDICTION_TOPIC = kafka_topic

    #FUNCIONAL
    for row in prediction_features.toJSON().collect():
        print((row))
        producer.send(PREDICTION_TOPIC , row.encode())
        producer.flush()
    
    time.sleep(sleep)
    c = c + 1
    seed = seed + 1

{"version":"1.0","time":"1616630975","id":"f0c48ba4-387d-11ea-a137-2e728ce88126","type":"RF","event":"DATA","data":{"time":"1599497307","signal":"-58","freq":"433.78","mod":"OOK","payload":"e8e8e8e88888e8e888e8e88800"}}
{"version":"1.0","time":"1616630975","id":"f0c48ba4-387d-11ea-a137-2e728ce88126","type":"RF","event":"DATA","data":{"time":"1599497313","signal":"-55","freq":"433.78","mod":"OOK","payload":"e8e8e8e88888e8e888e8e88800"}}
{"version":"1.0","time":"1616630975","id":"f0c48ba4-387d-11ea-a137-2e728ce88126","type":"RF","event":"DATA","data":{"time":"1599497326","signal":"-51","freq":"433.78","mod":"OOK","payload":"88e8e8e8eee8e8eee8e8e8e800"}}
{"version":"1.0","time":"1616630975","id":"f0c48ba4-387d-11ea-a137-2e728ce88126","type":"RF","event":"DATA","data":{"time":"1599497342","signal":"-57","freq":"433.78","mod":"OOK","payload":"88e8e8e888a8e8eee8e8e8e800"}}
{"version":"1.0","time":"1616630975","id":"f0c48ba4-387d-11ea-a137-2e728ce88126","type":"RF","event":"DATA","data":{"tim

KeyboardInterrupt: 

In [ ]:
sc.stop()